# w_03 | Seaborn

* Vamos a recorrer sistemáticamente las distintas categorías de la API de Seaborn.
* Traemos un dataset e importamos las librerías necesarias
    * Todo Seaborn se suele usar desde el módulo principal que se le asigna el alias `sns`

In [2]:
import pandas as pd
import seaborn as sns
import numpy as np

inc = pd.read_csv("https://catalogodatos.gub.uy/dataset/239103c3-a3f4-4d3c-93f6-008198eeda62/resource/e45a435f-0968-4868-9759-2038840b836d/download/inc-ofrecimientostierra-datos-v20210804.csv", sep=";", decimal=",")
inc["Fecha"] = pd.to_datetime(inc["Fecha ingreso trámite"], errors="coerce")
inc[["Departamento", "aux"]] = inc["Padrones"].str.split("->", expand=True).iloc[:, :2]
inc["Rural"] = inc["aux"].str.split(":", expand=True).iloc[:, 0]
inc.drop(["aux", "Tipo de negocio.1", "aux", "Fecha ingreso trámite", "Padrones", "Multa por incumplimiento"], axis=1, inplace=True)
inc = inc.loc[inc["Moneda"] == "U$S", :]
inc = inc.loc[inc["Fecha"] > "2008-01-01", :]
inc.rename({"Precio total": "Precio", "Superficie total(ha)": "Sup", "Superficie CONEAT 100 (ha)": "Sup CONEAT"}, inplace=True, axis=1)
inc["Sup CONEAT"].replace(0, np.nan, inplace=True)
inc["Precio"] = inc["Precio"] / 1000
inc["Calidad"] = inc["Sup CONEAT"] / inc["Sup"]
inc["Calidad"] = inc["Calidad"].where(inc["Calidad"] < 5, pd.NA)
inc["PPH"] = inc["Precio"] / inc["Sup"] * 1000
inc.head()

ParserError: Error tokenizing data. C error: Expected 17 fields in line 50, saw 19


In [ ]:
inc.shape

In [ ]:
inc.info()

* `sns.scatterplot()` in `sns.lineplot()` muestran relaciones entre variables continuas.
* Continua refiere a que no hay saltos discretos en la distribución de la variable.

In [ ]:
sns.set_style(style="darkgrid", rc={"figure.figsize": (12, 6)})

sns.lineplot(data=inc, x="Fecha", y="Precio")

* Notar 3 parámetros clave
    * `data` refiere a los datos a graficar. Esto en general debería ser un `pandas.DataFrame`, pero pero puede ser un `dict`, numpy arrays o incluso listas.
    * `x` e `y` refiere a qué graficar en cada eje.
        * Se refiere a ellos por el nombre de la columna en el caso de que `data` sea un df.
    * Seaborn asume que la data es wide-form cuando `x` e `y` no están asignados.

* Cuando hay varios valores para un mismo `x`,  Seaborn las agrega y muestra un intervalo de confianza.

In [ ]:
sns.lineplot(data=inc.loc[inc["Fecha"] > "2020-01-01"], x="Fecha", y="Precio")

* También se puede definir que en lugar de mostrar un intervalo de confianza se muestre el desvío estándar.

In [ ]:
sns.lineplot(data=inc.loc[inc["Fecha"] > "2020-01-01"], x="Fecha", y="Precio", errorbar='sd' )

* Podemos definir cómo agrega observaciones.
    * O que no las agregue con `estimator=None`

El parámetro estimator en la función sns.lineplot() de Seaborn controla cómo se calcula y muestra el valor central de la distribución de los datos cuando estás trazando una línea. En otras palabras, determina cómo se representa la tendencia general de los datos en el gráfico de líneas.

In [ ]:
sns.lineplot(data=inc.loc[inc["Fecha"] > "2020-01-01"], x="Fecha", y="Precio", estimator="sum")

* Por defecto `x` e `y` son ordenados antes de graficar.
    * El ordenamiento de los ejes se puede desactivar.Esto significa que los puntos se conectarán en el orden en que aparecen en el conjunto de datos, sin importar si están en orden cronológico o no

In [ ]:
sns.lineplot(data=inc, x="Fecha", y="Precio", sort=False)

* Como vimos, `hue`, `style` y `size` permiten agregar dimensiones.
* En el siguiente paso pasamos a usar `seaborn.scatterplot()`.

* `hue="Departamento"`: Esta opción permite codificar una tercera variable categórica ("Departamento") utilizando colores diferentes para cada categoría. Cada punto en el gráfico estará coloreado según el valor de la columna "Departamento".

* `size="Estado de ocupación"`: Esta opción permite ajustar el tamaño de los puntos en función de otra variable categórica ("Estado de ocupación"). Los puntos tendrán tamaños diferentes según el valor de esta columna.

* `style="Tipo de negocio"`: Esto permite asignar diferentes estilos a los puntos en función de otra variable categórica ("Tipo de negocio"). Los puntos pueden tener formas diferentes para cada valor de esta columna.

In [ ]:
sns.scatterplot(data=inc, x="Sup", y="Precio", hue="Departamento",
                size="Estado de ocupación", style="Tipo de negocio")

* Esta leyenda es difícil de ubicar porque tiene muchas categorías.
* En estos casos podemos optar por moverla manualmente, o usar funciones a nivel de figura.
    * En la siguiente celda usamos `seaborn.relplot`, que es la función de figuras correspondiente a gráficos relacionales.
    * Por defecto la leyenda se ubica por fuera de la gráfica.
    * Las funciones de figura permiten definir `height` y `aspect` para controlar las dimensiones.

El parámetro aspect está configurado como 1.2, lo que significa que el ancho del gráfico será 1.2 veces más grande que su altura

In [ ]:
sns.relplot(data=inc, x="Sup", y="Precio", hue="Departamento",
                size="Estado de ocupación", style="Tipo de negocio", kind="scatter",height=8, aspect=1.2)

* Para mover la leyenda vamos a usar MPL.
    * Podemos usar ambas interfaces.

In [ ]:
import matplotlib.pyplot as plt

g = sns.scatterplot(data=inc, x="Sup", y="Precio", hue="Departamento",
                size="Estado de ocupación", style="Tipo de negocio")
g.legend(loc="upper left", bbox_to_anchor=(0, -0.1), ncol=4)

In [ ]:
import matplotlib.pyplot as plt

sns.scatterplot(data=inc, x="Sup", y="Precio", hue="Departamento",
                size="Estado de ocupación", style="Tipo de negocio")
plt.legend(loc="upper left", bbox_to_anchor=(0, -0.1), ncol=4)

* Funciones de nivel de eje (`axes-level functions`): Estas funciones, como `sns.scatterplot()`, `sns.lineplot()`, etc., devuelven un objeto de tipo "ejes" que representa un solo eje del gráfico. Puedes acceder directamente a los métodos y propiedades del objeto "ejes" para personalizar el gráfico de manera detallada. Por ejemplo, puedes agregar leyendas, títulos y personalizar los ejes.

* Funciones de nivel de figura (`figure-level functions`): Estas funciones, como `sns.relplot()`, sns.catplot(), etc., crean una figura completa que contiene uno o más ejes. Estas funciones generalmente tienen un alto nivel de automatización y permiten crear tipos de gráficos más complejos. Sin embargo, debido a esta automatización, a veces ciertas personalizaciones detalladas no son directamente accesibles.

* Notar que si llamamos una `figure-level function`, los axes no están disponibles a primer nivel. Esto quiere decir que cuando usas una función de nivel de figura como `sns.relplot()`, los objetos de "ejes" no están directamente disponibles a nivel superior (primer nivel). En otras palabras, no puedes acceder y personalizar los ejes individualmente como lo harías con las funciones de nivel de eje.

In [ ]:
import traceback

g = sns.relplot(data=inc, x="Sup", y="Precio", hue="Departamento",
                size="Estado de ocupación", style="Tipo de negocio", kind="scatter", height=8, aspect=1.2)
try:
    g.legend(loc="upper left", bbox_to_anchor=(0, -0.1), ncol=4)
except TypeError:
    traceback.print_exc()

* Pero podemos acceder a los axes para hacer modificaciones.
    * En este ejemplo mostramos como acceder al axes y definir una nueva leyenda.
    * Para que no queden leyendas duplicadas definimos `legend=False` en `relplot`, pero eso nos hace perder los datos de las leyendas.

In [ ]:
g = sns.relplot(data=inc, x="Sup", y="Precio", hue="Departamento",
                size="Estado de ocupación", style="Tipo de negocio", kind="scatter", height=8, aspect=1.2, legend=False)
g.ax.legend(["Prueba de leyenda"], loc="upper left", bbox_to_anchor=(0, -0.1), ncol=4)

* Cuando usamos figure-level functions podemos especificar qué tipo de gráfico hacer con el parámetro `kind`, como `kind=scatter`.

* Es posible definir los colores con `palette`. Acepta algunos strings dados, y colormaps de MPL.

In [ ]:
sns.relplot(data=inc, x="Sup", y="Precio", hue="Departamento",
                size="Estado de ocupación", style="Tipo de negocio", kind="scatter", height=8, aspect=1.2, palette="tab20c")

* Los gráficos a nivel de axes aceptan un parámetro `ax`, lo que implica que podemos combinarlo con MPL fácilmente.

In [ ]:
fig, axs = plt.subplots(1, 3)
for ax, estado in zip(axs, inc["Estado de ocupación"].unique()):
    sns.lineplot(ax=ax, data=inc.loc[inc["Estado de ocupación"] == estado, :], x="Fecha", y="Precio")
    ax.set_title(estado)

* Pero muchas veces es más fácil usar `col` con figure-level functions.
* `col_wrap` define el máximo de gráficos por fila.
* Hay parámetros análogos para filas.

In [ ]:
sns.relplot(data=inc, x="Fecha", y="Precio", col="Estado de ocupación", col_wrap=4, kind="line",height=4)

* Así como podemos graficar relaciones entre variables, podemos verificar distribuciones.
* Eso se hace con las funciones `displot()` (a nivel de figura, y con la posibilidad de armar grids), `histplot()` para histogramas y `kdeplot()` para kernel distributions (como un histograma).


In [ ]:
sns.displot(data=inc, x="Precio", kind="hist", hue="Estado de ocupación", col="Departamento", col_wrap=4)

* Finalmente, las funciones `catplot()` se usan para gráficar cuando alguna de los ejes tiene variables categóricas.
* En este grupo entran `barplot()`, `boxplot()` y `violinplot`.

* `countplot()` merece una mención especial, dado que funciona como un gráfico de barras que automáticamente cuenta observaciones.

In [ ]:
sns.countplot(data=inc, x="Estado de ocupación")

In [ ]:
sns.violinplot(data=inc, x="Estado de ocupación", y="Precio")

* La fortaleza de Seaborn es que nos permite cambiar entre tipos de gráficas muy rápido sin cambiar los parámetros.

In [ ]:
sns.boxplot(data=inc, x="Estado de ocupación", y="Precio")

* `pairplot()` es un tipo de gráfica que no entra en la lógica de las anteriores, pero ayuda a reconocer un dataset muy rápido.

In [ ]:
sns.pairplot(data=inc, hue="Estado de ocupación")

# Fin